In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

**Source Dataset**

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
classes = ["a09", "a13", "a14", "a15", "a07", "a03"]
persons = ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"]
x_train_source = []
y_train_source = []
for cl in range(6):
  for p in persons:
    for s in range(60):
      if s+1 < 10:
        link = "drive/My Drive/"+classes[cl]+"/"+p+"/s0"+str(s+1)+".txt"
      else:
        link = "drive/My Drive/"+classes[cl]+"/"+p+"/s"+str(s+1)+".txt"
      f = open(link, "r")
      data_point = []
      for x in f:
        ax = [float(i) for i in x.split(',')[:6]]
        data_point.append(ax)
      x_train_source.append(data_point)
      y_train_source.append(cl+1)

In [ ]:
x_train_source = torch.from_numpy(np.array(x_train_source)).reshape(2880,125,6).type(torch.FloatTensor)
y_train_source = torch.from_numpy(np.array(y_train_source)).type(torch.FloatTensor)

**Target Dataset**

In [ ]:
bax = torch.from_numpy(np.loadtxt("body_acc_x_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bay = torch.from_numpy(np.loadtxt("body_acc_y_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
baz = torch.from_numpy(np.loadtxt("body_acc_z_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgx = torch.from_numpy(np.loadtxt("body_gyro_x_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgy = torch.from_numpy(np.loadtxt("body_gyro_y_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
bgz = torch.from_numpy(np.loadtxt("body_gyro_z_train.txt")[:,:125]).reshape(7352,125,1).type(torch.FloatTensor)
x_train = torch.cat((bax, bay, baz, bgx, bgy, bgz), 2)
y_train = torch.from_numpy(np.loadtxt("y_train.txt")).type(torch.FloatTensor) 

x_train = x_train[int(len(x_train)/2):]
y_train = y_train[int(len(y_train)/2):]


bax_test = torch.from_numpy(np.loadtxt("body_acc_x_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bay_test = torch.from_numpy(np.loadtxt("body_acc_y_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
baz_test = torch.from_numpy(np.loadtxt("body_acc_z_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgx_test = torch.from_numpy(np.loadtxt("body_gyro_x_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgy_test = torch.from_numpy(np.loadtxt("body_gyro_y_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
bgz_test = torch.from_numpy(np.loadtxt("body_gyro_z_test.txt")[:,:125]).reshape(2947,125,1).type(torch.FloatTensor)
x_test = torch.cat((bax_test, bay_test, baz_test, bgx_test, bgy_test, bgz_test), 2)
y_test = torch.from_numpy(np.loadtxt("y_test.txt")).type(torch.FloatTensor) 

In [ ]:
class LSTM(nn.Module):
  def __init__(self, n_input, n_hidden,
                 n_classes, drop_prob):
        super(LSTM, self).__init__()

        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.drop_prob = drop_prob
        self.n_input = n_input

        self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        #self.batch = nn.BatchNorm2d()
        self.drop1 = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(n_hidden, 1)
        self.fc2 = nn.Linear(n_hidden, n_classes)
        self.activation = nn.Softmax(dim=1)

  def forward(self, x):
        out, (hn,cn) = self.lstm1(x)
        out, (hn,cn) = self.lstm2(out)
        out, (hn,cn) = self.lstm3(out)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

  def init_hidden(self, batch_size):
        ''' Initialize hidden state'''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        # if (train_on_gpu):
        if (torch.cuda.is_available() ):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden


In [ ]:
def accuracy(output, target):
  preds = torch.argmax(output, dim = 1)
  c = 0
  for i in range(len(preds)):
    if preds[i] == target[i]:
      c += 1
  return c/len(preds)

In [ ]:
from sklearn.metrics import accuracy_score

def train(model, X, Y, num_epochs, num_classes, batch_size):
  criterion = nn.NLLLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=.001)
  #data = []
  #for i in range(len(X)):
  #  data.append([X[i], Y[i]])
  dataset = torch.utils.data.TensorDataset(torch.Tensor(np.array(X)), torch.Tensor(np.array(Y)))
  train_set_size = int(len(dataset)*0.8)
  val_set_size = len(dataset) - train_set_size
  train_set, val_set = torch.utils.data.random_split(dataset, [train_set_size, val_set_size])
  trainloader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=100)
  valloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size=100)

  for epoch in range(num_epochs):
    step_loss = 0
    train_acc = []
    model.train()
    for step, (x, y) in enumerate(trainloader):
      optimizer.zero_grad()
      outputs = model.forward(x.float())
      loss = criterion(torch.log(outputs), y.long()-1)
      train_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      loss.backward() #calculates the loss of the loss function
      optimizer.step()
      step_loss += loss
    print("Training Loss: ",step_loss.item())
    print("Training accuracy: ",np.sum(train_acc)/train_set_size)

    model.eval()
    val_loss = 0
    val_acc = []
    for step, (x, y) in enumerate(valloader):
      outputs = model.forward(x.float())
      loss = criterion(torch.log(outputs), y.long()-1)
      val_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      val_loss += loss
    print("Validation Loss: ",val_loss.item())
    print("Validation accuracy: ",np.sum(val_acc)/val_set_size)
  return model


In [ ]:

#model = train(x_train_source, y_train_source, 20, 6, 100)
pre_trained_model = LSTM(6,125,6,.3)
pre_trained_model = train(pre_trained_model, x_train_source, y_train_source, 20, 6, 100)

Training Loss:  37.36975860595703
Training accuracy:  0.30078125
Validation Loss:  8.422944068908691
Validation accuracy:  0.3385416666666667
Training Loss:  32.06546401977539
Training accuracy:  0.3424479166666667
Validation Loss:  7.68480920791626
Validation accuracy:  0.3784722222222222
Training Loss:  31.327342987060547
Training accuracy:  0.3793402777777778
Validation Loss:  7.476946830749512
Validation accuracy:  0.4270833333333333
Training Loss:  29.910112380981445
Training accuracy:  0.4210069444444444
Validation Loss:  7.418369770050049
Validation accuracy:  0.3854166666666667
Training Loss:  28.981046676635742
Training accuracy:  0.4288194444444444
Validation Loss:  6.929805755615234
Validation accuracy:  0.4826388888888889
Training Loss:  25.13368797302246
Training accuracy:  0.484375
Validation Loss:  6.146280288696289
Validation accuracy:  0.4947916666666667
Training Loss:  22.57573890686035
Training accuracy:  0.5720486111111112
Validation Loss:  5.15699577331543
Validati

Spottune

In [ ]:
class Agent(nn.Module):
  def __init__(self, n_input, n_hidden, n_output, drop_prob):
    super(Agent, self).__init__()

    self.n_input = n_input
    self.n_output = n_output
    self.n_hidden = n_hidden
    self.drop_prob = drop_prob

    self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
    self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
    #self.batch = nn.BatchNorm2d()
    self.drop1 = nn.Dropout(drop_prob)
    self.fc1 = nn.Linear(n_hidden, 1)
    self.fc2 = nn.Linear(n_hidden, n_output)
    self.activation = nn.Softmax(dim=1)

  def forward(self, x):
        out, (hn,cn) = self.lstm1(x)
        out, (hn, cn) = self.lstm2(out)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

  def init_hidden(self, batch_size):
        ''' Initialize hidden state'''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        # if (train_on_gpu):
        if (torch.cuda.is_available() ):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden  

In [ ]:
class SpottuneLSTM(nn.Module):
  def __init__(self, n_input, n_hidden,
                 n_classes, drop_prob):
        super(SpottuneLSTM, self).__init__()

        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.drop_prob = drop_prob
        self.n_input = n_input

        self.lstm1 = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3 = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)

        self.lstm1_frozen = nn.LSTM(n_input, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm2_frozen = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)
        self.lstm3_frozen = nn.LSTM(n_hidden, n_hidden, 1, dropout=self.drop_prob, batch_first=True)


        #self.batch = nn.BatchNorm2d()
        self.drop1 = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(n_hidden, 1)
        self.fc2 = nn.Linear(n_hidden, n_classes)
        self.activation = nn.Softmax(dim=1)
  def lstm_output(self, lstm, lstm_frozen, policy, x, n_input):
    for i in range(len(policy)):
      ax = torch.reshape(x[i], (1, 125, n_input))
      if policy[i] == 0.:
        out_i, (_, _) = lstm_frozen(ax)
      else:
        out_i, (_, _) = lstm(ax)
      if i == 0:
        out = out_i
      else:
        out = torch.cat((out, out_i), 0)
    return out
  def forward(self, x, policy = None):
        out = self.lstm_output(self.lstm1, self.lstm1_frozen, policy[:, 0], x, self.n_input)
        #out, (hn,cn) = policy[:,0]*self.lstm1(x)+(1-policy[:,0])*self.lstm1_frozen(x)
        #out, (hn,cn) = policy[:,1]*self.lstm2(x)+(1-policy[:,1])*self.lstm2_frozen(x)
        out = self.lstm_output(self.lstm2, self.lstm2_frozen, policy[:, 1], out, self.n_hidden)
        #out, (hn,cn) = policy[:,2]*self.lstm3(x)+(1-policy[:,2])*self.lstm3_frozen(x)
        out = self.lstm_output(self.lstm3, self.lstm3_frozen, policy[:, 2], out, self.n_hidden)
        out = self.fc1(self.drop1(out))
        #out = self.fc2(self.drop1(out[:,:,0]))
        #out = self.fc1(out)
        out = self.fc2(out[:,:,0])
        out = self.activation(out)
        return out #(torch.argmax(out, dim = 1) + 1)

In [ ]:
##It's working, it ust needed time and I wanted to go to sleep. Goodnight!!

spottunemodel = SpottuneLSTM(6,125,6,.3)
spottunemodel.lstm1_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm1_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm1_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm1_frozen.bias_hh_l0.requires_grad = False

spottunemodel.lstm2_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm2_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm2_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm2_frozen.bias_hh_l0.requires_grad = False

spottunemodel.lstm3_frozen.weight_ih_l0.requires_grad = False
spottunemodel.lstm3_frozen.weight_hh_l0.requires_grad = False
spottunemodel.lstm3_frozen.bias_ih_l0.requires_grad = False
spottunemodel.lstm3_frozen.bias_hh_l0.requires_grad = False
states = pre_trained_model.state_dict()
states["lstm1_frozen.weight_ih_l0"] = states["lstm1.weight_ih_l0"]
states["lstm1_frozen.weight_hh_l0"] = states["lstm1.weight_hh_l0"]
states["lstm1_frozen.bias_ih_l0"] = states["lstm1.bias_ih_l0"]
states["lstm1_frozen.bias_hh_l0"] = states["lstm1.bias_hh_l0"]
states["lstm2_frozen.weight_ih_l0"] = states["lstm2.weight_ih_l0"]
states["lstm2_frozen.weight_hh_l0"] = states["lstm2.weight_hh_l0"]
states["lstm2_frozen.bias_ih_l0"] = states["lstm2.bias_ih_l0"]
states["lstm2_frozen.bias_hh_l0"] = states["lstm2.bias_hh_l0"]
states["lstm3_frozen.weight_ih_l0"] = states["lstm3.weight_ih_l0"]
states["lstm3_frozen.weight_hh_l0"] = states["lstm3.weight_hh_l0"]
states["lstm3_frozen.bias_ih_l0"] = states["lstm3.bias_ih_l0"]
states["lstm3_frozen.bias_hh_l0"] = states["lstm3.bias_hh_l0"]
spottunemodel.load_state_dict(states)

def spottunetrain(model, agent, X, Y, num_epochs, num_classes, batch_size):
  criterion = nn.NLLLoss()
  optimizer_model = torch.optim.Adam(model.parameters(), lr=.001)
  optimizer_agent = torch.optim.Adam(agent.parameters(), lr=.001)
  dataset = torch.utils.data.TensorDataset(torch.Tensor(np.array(X)), torch.Tensor(np.array(Y)))
  train_set_size = int(len(dataset)*0.8)
  val_set_size = len(dataset) - train_set_size
  train_set, val_set = torch.utils.data.random_split(dataset, [train_set_size, val_set_size])
  trainloader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=100)
  valloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size=100)

  for epoch in range(num_epochs):
    model.train()
    step_loss = 0
    train_acc = []
    for step, (x, y) in enumerate(trainloader):
      optimizer_model.zero_grad()
      optimizer_agent.zero_grad()
      policy = agent(x)
      action = F.gumbel_softmax(policy, tau=1, hard=True)
      outputs = model.forward(x.float(), action)
      loss = criterion(torch.log(outputs), y.long()-1)
      train_acc.append(accuracy(outputs,y-1)*len(x))
      #loss.requires_grad = True
      loss.backward() #calculates the loss of the loss function
      if action[0][0] == 0:
        if spottunemodel.lstm1.weight_ih_l0.grad is not None:
          spottunemodel.lstm1.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm1.weight_hh_l0.grad is not None:
          spottunemodel.lstm1.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm1.bias_ih_l0.grad is not None:
          spottunemodel.lstm1.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm1.bias_hh_l0.grad is not None:
          spottunemodel.lstm1.bias_hh_l0.grad.data.zero_()
      if action[0][1] == 0:
        if spottunemodel.lstm2.weight_ih_l0.grad is not None:
          spottunemodel.lstm2.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm2.weight_hh_l0.grad is not None:
          spottunemodel.lstm2.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm2.bias_ih_l0.grad is not None:
          spottunemodel.lstm2.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm2.bias_hh_l0.grad is not None:
          spottunemodel.lstm2.bias_hh_l0.grad.data.zero_()
      if action[0][2] == 0:
        if spottunemodel.lstm3.weight_ih_l0.grad is not None:
          spottunemodel.lstm3.weight_ih_l0.grad.data.zero_()
        if spottunemodel.lstm3.weight_hh_l0.grad is not None:
          spottunemodel.lstm3.weight_hh_l0.grad.data.zero_()
        if spottunemodel.lstm3.bias_ih_l0.grad is not None:
          spottunemodel.lstm3.bias_ih_l0.grad.data.zero_()
        if spottunemodel.lstm3.bias_hh_l0.grad is not None:
          spottunemodel.lstm3.bias_hh_l0.grad.data.zero_()
      optimizer_model.step()
      optimizer_agent.step()
      step_loss += loss
    print("Training Loss: ",step_loss.item())
    print("Training Accuracy: ",np.sum(train_acc)/train_set_size)

    model.eval()
    val_loss = 0
    val_acc = []
    for step, (x, y) in enumerate(valloader):
      policy = agent(x)
      action = F.gumbel_softmax(policy, tau=1, hard=True)
      outputs = model.forward(x.float(), action)
      loss = criterion(torch.log(outputs), y.long()-1)
      val_acc.append(accuracy(outputs,y-1)*len(x))
      val_loss += loss
    print("Validation Loss: ",val_loss.item())
    print("Validation Accuracy: ",np.sum(val_acc)/val_set_size)
  return model

agent = Agent(6, 125, 3, 0.3)
#spottunemodel = spottunetrain(spottunemodel, agent, x_train, y_train, 20, 6, 100)
spottunemodel = spottunetrain(spottunemodel, agent, x_train, y_train, 5, 6, 100)

NameError: ignored

In [ ]:
testset = torch.utils.data.TensorDataset(torch.Tensor(np.array(x_test)), torch.Tensor(np.array(y_test)))
testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=len(testset))
criterion = nn.NLLLoss()
spottunemodel.eval()
test_loss = 0
test_acc = []
for step, (x, y) in enumerate(testloader):
  policy = agent(x)
  action = F.gumbel_softmax(policy, tau=1, hard=True)
  outputs = spottunemodel.forward(x.float(), action)
  loss = criterion(torch.log(outputs), y.long()-1)
  test_acc.append(accuracy(outputs,y-1)*len(x))
  test_loss += loss
print("Validation Loss: ",test_loss.item())
print("Validation Accuracy: ",np.sum(test_acc)/len(x_test))

**Benchmark**

In [ ]:
benchmark_model = LSTM(6,125,6,.3)
benchmark_model = train(benchmark_model, x_train, y_train, 20, 6, 100)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Training Loss:  53.57286834716797
Training accuracy:  0.18333333333333332
Validation Loss:  14.223167419433594
Validation accuracy:  0.20108695652173914


KeyboardInterrupt: ignored

In [ ]:
testset = torch.utils.data.TensorDataset(torch.Tensor(np.array(x_test)), torch.Tensor(np.array(y_test)))
testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=len(testset))
criterion = nn.NLLLoss()
benchmark_model.eval()
test_loss = 0
test_acc = []
for step, (x, y) in enumerate(testloader):
  outputs = benchmark_model.forward(x.float())
  loss = criterion(torch.log(outputs), y.long()-1)
  test_acc.append(accuracy(outputs,y-1)*len(x))
  test_loss += loss
print("Test Loss: ",test_loss.item())
print("Test Accuracy: ",np.sum(test_acc)/len(x_test))

Test Loss:  1.095755934715271
Test Accuracy:  0.45062775704105873


#1
Spottune:<br><br>
Training Loss:  200.94908142089844<br>
Training Accuracy:  0.1435374149659864<br>
Validation Loss:  16.344161987304688<br>
Validation Accuracy:  0.13722826086956522<br><br>
Training Loss:  55.85991287231445<br>
Training Accuracy:  0.18333333333333332<br>
Validation Loss:  14.765766143798828<br>
Validation Accuracy:  0.21603260869565216<br><br>
Training Loss:  54.13177490234375<br>
Training Accuracy:  0.2030612244897959<br>
Validation Loss:  14.588086128234863<br>
Validation Accuracy:  0.22010869565217392<br><br>
Training Loss:  53.89824295043945<br>
Training Accuracy:  0.20510204081632652<br>
Validation Loss:  14.438674926757812<br>
Validation Accuracy:  0.22146739130434784<br><br>
Training Loss:  53.751136779785156<br>
Training Accuracy:  0.21360544217687075<br>
Validation Loss:  14.462971687316895<br>
Validation Accuracy:  0.2105978260869565<br><br>
Test Loss:  1.7950762510299683<br>
Test Accuracy:  0.17712928401764505<br>
<br>
Regular LSTM:<br><br>
Training Loss:  53.55483627319336<br>
Training accuracy:  0.19795918367346937<br>
Validation Loss:  14.216048240661621<br>
Validation accuracy:  0.19293478260869565<br><br>
Training Loss:  46.793758392333984<br>
Training accuracy:  0.2693877551020408<br>
Validation Loss:  9.893157005310059<br>
Validation accuracy:  0.28940217391304346<br><br>
Training Loss:  35.88084030151367<br>
Training accuracy:  0.34251700680272107<br>
Validation Loss:  9.668484687805176<br>
Validation accuracy:  0.3383152173913043<br><br>
Training Loss:  35.31169891357422<br>
Training accuracy:  0.3476190476190476<br>
Validation Loss:  9.428119659423828<br>
Validation accuracy:  0.3451086956521739<br><br>
Training Loss:  34.295257568359375<br>
Training accuracy:  0.37755102040816324<br>
Validation Loss:  9.395288467407227<br>
Validation accuracy:  0.34375<br><br>
Test Loss:  1.1956709623336792<br>
Test Accuracy:  0.34781133355955207<br>

#2
Spottune<br><br>
Training Loss:  83.13389587402344<br>
Training Accuracy:  0.1523809523809524<br>
Validation Loss:  14.668261528015137<br>
Validation Accuracy:  0.17798913043478262<br><br>
Training Loss:  53.87263870239258<br>
Training Accuracy:  0.19115646258503402<br>
Validation Loss:  14.242119789123535<br>
Validation Accuracy:  0.20244565217391305<br><br>
Training Loss:  53.283512115478516<br>
Training Accuracy:  0.204421768707483<br>
Validation Loss:  14.249770164489746<br>
Validation Accuracy:  0.19293478260869565<br><br>
Training Loss:  53.08428192138672<br>
Training Accuracy:  0.20204081632653062<br>
Validation Loss:  14.192403793334961<br>
Validation Accuracy:  0.1983695652173913<br><br>
Training Loss:  53.044898986816406<br>
Training Accuracy:  0.19421768707482992<br>
Validation Loss:  14.245340347290039<br>
Validation Accuracy:  0.2078804347826087<br>